In [1]:
from main_2 import main, Config

In [2]:
emsize_nhid_values = [200, 400, 600]  # Example values
nlayers_values = [1, 2, 3]

In [4]:
results = {}
config = Config()
config.epochs = 5 #shorten # of epoch for time efficiency
for emsize_nhid in emsize_nhid_values:
    for nlayers in nlayers_values:
        config.emsize = emsize_nhid
        config.nhid = emsize_nhid
        config.nlayers = nlayers

        if emsize_nhid == 200 and nlayers == 2:
            continue  # Skip this iteration - already have results from part 1
        print(f"Running experiment for: emsize_nhid:  {emsize_nhid}, nlayers: {nlayers}")
        test_ppl = main(config)
        results[(emsize_nhid, nlayers)] = test_ppl


Running experiment for: emsize_nhid:  200, nlayers: 1
| epoch   1 |   200/ 2983 batches | lr 20.00 | ms/batch 587.85 | loss  7.52 | ppl  1836.87
| epoch   1 |   400/ 2983 batches | lr 20.00 | ms/batch 596.20 | loss  6.63 | ppl   760.79
| epoch   1 |   600/ 2983 batches | lr 20.00 | ms/batch 584.64 | loss  6.27 | ppl   528.86
| epoch   1 |   800/ 2983 batches | lr 20.00 | ms/batch 594.88 | loss  6.12 | ppl   455.86
| epoch   1 |  1000/ 2983 batches | lr 20.00 | ms/batch 592.22 | loss  6.00 | ppl   404.27
| epoch   1 |  1200/ 2983 batches | lr 20.00 | ms/batch 626.19 | loss  5.95 | ppl   382.47
| epoch   1 |  1400/ 2983 batches | lr 20.00 | ms/batch 534.20 | loss  5.86 | ppl   350.73
| epoch   1 |  1600/ 2983 batches | lr 20.00 | ms/batch 502.60 | loss  5.87 | ppl   355.71
| epoch   1 |  1800/ 2983 batches | lr 20.00 | ms/batch 496.87 | loss  5.74 | ppl   309.61
-----------------------------------------------------------------------------------------
Exiting from training early


AttributeError: 'TransformerModel' object has no attribute 'rnn'